# LESSON 2: Informed Search Strategies

In the second session we will work on informed search

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

### Priority Queue

You will need a queue ordered by priority as queue, or **PriorityQueue**. The difference between the other versions of queue is that in **PriorityQueue** nodes are removed from the queue based on the current lowest value. In particular, **Node** has two useful parameters (other than those used in the previous session):

- pathcost - the path cost from the root node to the current one (defaults to 0)
- value - value of a node. Used by PriorityQueue to order its content (defaults to 0)

Here is an example of usage:

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *

# Create 3 nodes for state ids 1 2 3
node_1 = Node(1) # No parent, pathcost=0, value=0
node_2 = Node(2, node_1, node_1.pathcost + 1, 10) # Child of node_1, pathcost=1, value=10
node_3 = Node(3, node_1, 100, 5)  # Child of node_1, pathcost=100, value=5

p_queue = PriorityQueue()
for n in (node_1, node_2, node_3):
    p_queue.add(n)
    print("Added: {}".format(n.state))

while not p_queue.is_empty():
    print("Removed: {}".format(p_queue.remove().state))

Added: 1
Added: 2
Added: 3
Removed: 1
Removed: 3
Removed: 2


Notice the order in which nodes are removed from the queue.

### Uniform-Cost Search (UCS)
Before moving to informed search it is useful to see another uninformed search algorithm: the Uniform-Cost Search (UCS). In the following you can see the implementation in graph search version. Cost of performing an action is supposd to be 1 (also in the assignements).

In [2]:
def present_with_higher_cost(queue, node):
    if (node.state in queue):
        if(queue[node.state].value > node.value): return True
    return False

In [3]:
import gym
import envs

def ucs(environment):
    """
    Uniform-cost search
    
    Args:
        environment: OpenAI Gym environment
        
    Returns:
        path: solution as a path
    """

    start_node = Node(environment.startstate)

    explored = set()
    frontier = PriorityQueue()
    frontier.add(start_node)
    
    time_cost  = 0
    space_cost = 1
    
    while not frontier.is_empty():
        current_node = frontier.remove()
        
        # Abbiamo finito
        if current_node.state == environment.goalstate: 
            return build_path(current_node), time_cost, space_cost
    
        explored.add(current_node.state)
        
        # Esploriamo gli stati successivi
        for action in range(environment.action_space.n):
            
            next_node_path_cost = current_node.pathcost + 1
            next_node_value = next_node_path_cost

            next_state = environment.sample(current_node.state, action)
            next_node = Node(next_state, current_node, next_node_path_cost, next_node_value)  
            
            # Se il nodo non è stato già visitato / sarà visitato
            if next_node.state not in frontier and next_node.state not in explored:
                frontier.add(next_node)

            # Se il nodo è presente con un costo maggiore
            elif present_with_higher_cost(frontier, next_node):
                frontier.replace(next_node)

            time_cost += 1
                
        space_cost = max(space_cost, len(frontier) + len(explored))

    return None

In [4]:
# Create and render the environment
env = gym.make("SmallMaze-v0")
env.render()
solution, time, memory = ucs(env)

print("\n----------------------------------------------------------------")
print("\nUNIFORM GRAPH SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Solution: {}".format(solution_2_string(solution, env)))
print("N° of nodes explored: {}".format(time))
print("Max n° of nodes in memory: {}".format(memory))

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]

----------------------------------------------------------------

UNIFORM GRAPH SEARCH PROBLEM: 
----------------------------------------------------------------
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 60
Max n° of nodes in memory: 16


### Distance Heuristics

Informed search requires a distance heuristic to be used in order to estimate the distance between a state and the goal. You already have at your disposal these functions:

- *l1_norm(p1, p2)* - Computes the L1 norm (also known as the manhattan distance) between two points specified as tuples of coordinates
- *l2_norm(p1, p2)* - Computes the L2 norm between two points specified as tuples of coordinates
- *chebyshev(p1, p2)* - Computes the Chebyshev distance between two points specified as tuples of coordinates. Similar to L1 norm but diagonal moves are also considered


Examples:

In [5]:
p1 = (0, 2)
p2 = (4, 0)
print("L1 norm heuristic value: {}".format(Heu.l1_norm(p1, p2)))
print("L2 norm heuristic value: {}".format(Heu.l2_norm(p1, p2)))
print("Chebyshev heuristic value: {}".format(Heu.chebyshev(p1, p2)))

L1 norm heuristic value: 6
L2 norm heuristic value: 4.47213595499958
Chebyshev heuristic value: 4


## Assignment 1: Greedy Best-First Search

The first assignment is to implement the Greedy-best-first search algorithm on **SmallMaze**. In particular, you are required to implement both *greedy_tree_search* and *greedy_graph_search* versions that will be called by the generic *greedy*. Use L1 norm as heuristic function at first, then try also the others to see the differences.

The results returned by greedy must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory at the same time.

Functions to implement:
- *greedy_tree_search(environment)*
- *greedy_graph_search(environment)*

In [17]:
def greedy_tree_search(environment, heuristic, timeout=10000):
    """
    Greedy-best-first Tree search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    start_node = Node(environment.startstate)
    goal_state_pos = environment.state_to_pos(environment.goalstate)

    frontier = PriorityQueue()
    frontier.add(start_node)

    time_cost  = 0
    space_cost = 1

    while not frontier.is_empty():
        
        # timeout check
        if time_cost >= timeout: 
            return "time-out", time_cost, space_cost
        
        # Se abbiamo raggiunto lo stato finale
        current_node = frontier.remove()
        if current_node.state == environment.goalstate:
            return build_path(current_node), time_cost, space_cost
        
        # Vediamo i possibili stati successori
        for action in range(environment.action_space.n):

            next_state = environment.sample(current_node.state, action)
            next_state_pos = environment.state_to_pos(next_state)

            next_node_path_cost = current_node.pathcost + 1
            next_node_value = heuristic(next_state_pos, goal_state_pos) # Euristica

            next_node = Node(next_state, current_node, next_node_path_cost, next_node_value)
            frontier.add(next_node)

            # Se il nodo è presente con un costo maggiore
            if present_with_higher_cost(frontier, next_node):
                frontier.replace(next_node)

            time_cost += 1
                
        space_cost = max(space_cost, len(frontier))
        
    return 'failure', time_cost, space_cost

In [18]:
def greedy_graph_search(environment, heuristic):
    """
    Greedy-best-first Graph search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    start_node = Node(environment.startstate)
    goal_state_pos = environment.state_to_pos(environment.goalstate)

    explored = set()
    frontier = PriorityQueue()
    frontier.add(start_node)

    time_cost  = 0
    space_cost = 1

    while not frontier.is_empty():
        
        # Se abbiamo raggiunto lo stato finale
        current_node = frontier.remove()
        if current_node.state == environment.goalstate:
            return build_path(current_node), time_cost, space_cost
        
        explored.add(current_node.state)

        # Vediamo i possibili stati successori
        for action in range(environment.action_space.n):

            next_state = environment.sample(current_node.state, action)
            next_state_pos = environment.state_to_pos(next_state)

            next_node_path_cost = current_node.pathcost + 1
            next_node_value = heuristic(next_state_pos, goal_state_pos) # Euristica

            # Se il nodo non è stato già visitato / sarà visitato
            next_node = Node(next_state, current_node, next_node_path_cost, next_node_value)
            if next_node.state not in frontier and next_node.state not in explored:
                frontier.add(next_node)

            # Se il nodo è presente con un costo maggiore
            elif present_with_higher_cost(frontier, next_node):
                frontier.replace(next_node)

            time_cost += 1
                
        space_cost = max(space_cost, len(frontier) + len(explored))
        
    return 'failure', time_cost, space_cost

The following function calls your implementations of greedy_tree_search and greedy_graph_search:

In [15]:
def greedy(environment, heuristic, search_type):
    """
    Greedy-best-first search
    
    Args:
        problem: OpenAI Gym environment
        search_type: type of search - greedy_tree_search or greedy_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    path, time_cost, space_cost = search_type(environment, heuristic)
    return path, time_cost, space_cost

The following code calls your tree search and graph search version of Greedy-best-first search and prints the results:

In [25]:
envname = "SmallMaze-v0"
environment = gym.make(envname)
heuristic = Heu.l2_norm

solution_ts, time_ts, memory_ts = greedy(environment, heuristic, greedy_tree_search)
solution_gs, time_gs, memory_gs = greedy(environment, heuristic, greedy_graph_search)

print("\n----------------------------------------------------------------")
print("\tGREEDY BEST FIRST TREE SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Solution: {}".format(solution_2_string(solution_ts, environment)))
print("N° of nodes explored: {}".format(time_ts))
print("Max n° of nodes in memory: {}".format(memory_ts))

print("\n----------------------------------------------------------------")
print("\tGREEDY BEST FIRST GRAPH SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Solution: {}".format(solution_2_string(solution_gs, environment)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}".format(memory_gs))


----------------------------------------------------------------
	GREEDY BEST FIRST TREE SEARCH PROBLEM: 
----------------------------------------------------------------
Solution: time-out
N° of nodes explored: 10000
Max n° of nodes in memory: 7501

----------------------------------------------------------------
	GREEDY BEST FIRST GRAPH SEARCH PROBLEM: 
----------------------------------------------------------------
Solution: [(0, 3), (1, 3), (2, 3), (2, 2), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 44
Max n° of nodes in memory: 15


Correct results can be found [here](lesson_2_results.txt).

## Assignment 2: A* Search
The second assignment is to implement the A search algorithm on SmallMaze. In particular, you are required to implement both astar_tree_search and astar_graph_search versions that will be called by the generic astar. Use L1 norm* as heuristic function at first, then try also the others to see the differences.

The results returned by astar must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory at the same time.

Functions to implement:
- *astar_tree_search(environment)*
- *astar_graph_search(environment)*

In [10]:
def astar_tree_search(environment):
    """
    A* Tree search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    start_node = Node(environment.startstate)
    goal_state_pos = environment.state_to_pos(environment.goalstate)

    frontier = PriorityQueue()
    frontier.add(start_node)

    time_cost  = 0
    space_cost = 1

    while not frontier.is_empty():
        
        # Se abbiamo raggiunto lo stato finale
        current_node = frontier.remove()
        if current_node.state == environment.goalstate:
            return build_path(current_node), time_cost, space_cost
        
        # Vediamo i possibili stati successori
        for action in range(environment.action_space.n):

            next_state = environment.sample(current_node.state, action)
            next_state_pos = environment.state_to_pos(next_state)

            next_node_path_cost = current_node.pathcost + 1
            next_node_value = next_node_path_cost + Heu.l1_norm(next_state_pos, goal_state_pos) # Euristica

            next_node = Node(next_state, current_node, next_node_path_cost, next_node_value)
            frontier.add(next_node)

            # Se il nodo è presente con un costo maggiore
            if present_with_higher_cost(frontier, next_node):
                frontier.replace(next_node)

            time_cost += 1
                
        space_cost = max(space_cost, len(frontier))
        
    return 'failure', time_cost, space_cost

In [11]:
def astar_graph_search(environment):
    """
    A* Graph search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    start_node = Node(environment.startstate)
    goal_state_pos = environment.state_to_pos(environment.goalstate)

    explored = set()
    frontier = PriorityQueue()
    frontier.add(start_node)

    time_cost  = 0
    space_cost = 1

    while not frontier.is_empty():
        
        # Se abbiamo raggiunto lo stato finale
        current_node = frontier.remove()
        if current_node.state == environment.goalstate:
            return build_path(current_node), time_cost, space_cost
        
        explored.add(current_node.state)

        # Vediamo i possibili stati successori
        for action in range(environment.action_space.n):

            next_state = environment.sample(current_node.state, action)
            next_state_pos = environment.state_to_pos(next_state)

            next_node_path_cost = current_node.pathcost + 1
            next_node_value = next_node_path_cost + Heu.l1_norm(next_state_pos, goal_state_pos) # Euristica

            # Se il nodo non è stato già visitato / sarà visitato
            next_node = Node(next_state, current_node, next_node_path_cost, next_node_value)
            if next_node.state not in frontier and next_node.state not in explored:
                frontier.add(next_node)

            # Se il nodo è presente con un costo maggiore
            elif present_with_higher_cost(frontier, next_node):
                frontier.replace(next_node)

            time_cost += 1
                
        space_cost = max(space_cost, len(frontier) + len(explored))
        
    return 'failure', time_cost, space_cost

The following function calls your implementations of astar_tree_search and astar_graph_search:

In [12]:
def astar(environment, search_type):
    """
    A* search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - astar_tree_search or astar_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    path, time_cost, space_cost = search_type(environment)
    return path, time_cost, space_cost

The following code calls your tree search and graph search version of A* search and prints the results:

In [13]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = astar(environment, astar_tree_search)
solution_gs, time_gs, memory_gs = astar(environment, astar_graph_search)

print("\n----------------------------------------------------------------")
print("\tA* TREE SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Solution: {}".format(solution_2_string(solution_ts, environment)))
print("N° of nodes explored: {}".format(time_ts))
print("Max n° of nodes in memory: {}".format(memory_ts))

print("\n----------------------------------------------------------------")
print("\tA* GRAPH SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Solution: {}".format(solution_2_string(solution_gs, environment)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}".format(memory_gs))


----------------------------------------------------------------
	A* TREE SEARCH PROBLEM: 
----------------------------------------------------------------
Solution: [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 8360
Max n° of nodes in memory: 6271

----------------------------------------------------------------
	A* GRAPH SEARCH PROBLEM: 
----------------------------------------------------------------
Solution: [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 60
Max n° of nodes in memory: 16


Correct results can be found [here](lesson_2_results.txt).

### Discussion
Now that you have correctly implemented both Greedy-best-first and A* what can you say about the solutions they compute? Are there significant differences in the stats? Try to play with other heuristics as well and see if your results change.